In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import csv
import nltk
from matplotlib import pyplot as plt
import os
import re
import warnings
import random
warnings.filterwarnings('ignore')
from collections import Counter,defaultdict
from nltk import word_tokenize
from math import log
import string
import pickle
from math import log
import gensim
from gensim import corpora
from gensim import models
from string import digits
from collections import defaultdict
import ast
from scipy.spatial.distance import cosine
cwd = os.getcwd()

In [2]:
osname = os.name
if osname =='Windows':
    sym = "//"
else:
    sym = "/"

In [3]:
r_path=cwd+sym+'nips_reviewer_data'+sym+'reviewers.txt'
review_df=pd.read_csv(r_path,sep='\t',header=None)
review_df.columns=['sno','name']
review_lt=list(review_df.name)
review_sno={val:ind for ind,val in enumerate(review_lt)}

## Loading the Data

In [4]:
path1=cwd+sym+'Saved_Items'+sym
with open(path1+'dictionary.pkl','rb') as f:
    dictionary=pickle.load(f)
with open(path1+'tfidf.pkl','rb') as f:
    tfidf=pickle.load(f)
with open(path1+'ldaModel','rb') as f:
    lda_model=pickle.load(f)
with open(path1+'paper_vec.pkl','rb') as f:
    paper_vec=pickle.load(f)
data_df=pd.read_pickle('data_df.pkl')

In [5]:
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
st = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
import re

def cleantext(text):
    
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    
    # remove punctuation
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    strip = text.translate(table)
    
    
    # Tokenizer
    tokens = word_tokenize(strip)
    
    # Convert into lower case 
    proc_text = [w.lower() for w in tokens]
    
    # Remove stopwords
    proc_text = [word for word in proc_text if word not in st]
    
    
    #Storing only Lemmmatized words
    lemmatizer=WordNetLemmatizer()
    lemma_text=[lemmatizer.lemmatize(word) for word in proc_text]

    return " ".join(lemma_text)


t = "i am groot2'7"    
cleantext(t)

'groot'

In [6]:
data_df.head()

,id,author,title,abstract,year,venue
0,53e99f9bb7602d970286e73d,Alan Murray,Adaptive Noisy Neural Computation in Mixed-mod...,We propose a probabilistic neural computation ...,2003,
1,53e99f9bb7602d9702870afd,Alan Murray,Geography 683 - Introduction to Geographic Ana...,,-1,
2,53e99f9bb7602d9702870cfb,David Forsyth,Achieve What We Want Them To? Interdependence ...,,-1,
3,53e99f9bb7602d9702871a1e,Ingo Steinwart,Fast Rates to Bayes for Kernel Machines,We establish learning rates to the Bayes risk ...,2004,NIPS
4,53e99f9bb7602d9702872b1d,David Forsyth,"Primitives, Perceptual Organization and Object...",: We argue that any computational theory of o...,1997,


## Computing reviewer Base LDA vectors

In [70]:
reviewer_vec_blda={}
for sno,name in enumerate(review_lt):
    samp_df=data_df[data_df['author']==name]
    index_lt=samp_df.index
    wts=np.zeros([250], dtype = float)
    ctr=0
    #print(index_lt)
    for ind in index_lt:
        ctr+=1
        temp_wt=np.array(paper_vec[ind])
        wts=wts+temp_wt
    wts=wts/ctr
    reviewer_vec_blda[sno]=wts
    
    

## Predicting the output

In [24]:
def predict(text,reviewer_vec_blda):
    clean_text=list(word_tokenize(cleantext(text))) 
    bow_text=dictionary.doc2bow(clean_text)
    tfidf_text=tfidf[bow_text]
    topic=lda_model[tfidf_text]
    
    topic_vec=np.zeros(250)
    for ind,wt in topic:
        topic_vec[ind]=wt
        
    #print(topic)
    
    reviewer_scr=[]
    index_lt=list(reviewer_vec_blda.keys())
    for index in index_lt:
        val=1-cosine(topic_vec,reviewer_vec_blda[index])
        if np.isnan(val):
            val=0
        reviewer_scr.append([index+1,val])
        
    reviewer_scr.sort(key=lambda x: x[1],reverse=True)
    
    pred=[]
    for i in range(10):
        pred.append(reviewer_scr[i][0])
        
    return pred

## Evaluation

In [10]:
path=path1+'Testing'+sym+'trec_judgments.txt'
ordinal=[]
value=[]
judge={}
with open(path,'r',encoding="utf8")as f:
    for line in f:
        ordinal=list(line.split('\t'))
        value=[int(ordinal[2]),int(ordinal[3][:-1])]
        if int(ordinal[0][4:]) in judge:
            judge[int(ordinal[0][4:])].append(value)
        else :
            judge[int(ordinal[0][4:])]=[value]
            
path=path1+'Testing'+sym+'trec_judgments_additional.txt'
with open(path,'r',encoding="utf8")as f:
    for line in f:
        ordinal=list(line.split('\t'))
        value=[int(ordinal[2]),int(ordinal[3][:-1])]
        if int(ordinal[0]) in judge:
            judge[int(ordinal[0])].append(value)
        else :
            judge[int(ordinal[0])]=[value]
            


In [59]:
def precision_k(res,gnd_tht,k,paper_no):
    actual_raw=gnd_tht[paper_no]
    #Getting tupules with unique first element
    y = np.unique(actual_raw, axis=0)
    actual = [] 
    for i in y:
        actual.append(tuple(i))
    
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)

    soft_p=0
    for i in range(k):
        predi=res[i]
        for j in range(n):
            if actual_auth[j]==predi:
                if rating[j]==3:
                    soft_p+=1
                elif rating[j]==2:
                    soft_p+=1
                elif rating[j]==1:
                    soft_p+=0
    soft_p=soft_p/k
    return soft_p

In [12]:
path1=cwd+sym+'Saved_Items'+sym
with open(path1+'ldaModel250','rb') as f:
    lda_model=pickle.load(f)

In [20]:
with open(path1+'reviewer_vec.pkl','rb') as f:
    reviewer_vec=pickle.load(f)

In [21]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avg_precision=[0]*11
valid_file=list(judge.keys())
for i in range(n):
    if i in valid_file:        
        res=predict(test_abs['abstract'].iloc[i],reviewer_vec)

        for k in range(1,11):
            avg_precision[k]+=precision_k(res,judge,k,i)
            
        ctr+=1
avg_precision=[ele/ctr for ele in avg_precision]

[(17, 0.0318619267886738), (21, 0.03829375773425805), (24, 0.013382444773253697), (50, 0.0541737918341939), (68, 0.012012634819293832), (84, 0.04400811624782172), (93, 0.01008973659961197), (94, 0.01279021598449918), (99, 0.018962355256592095), (114, 0.08104502274873421), (132, 0.03976908611658863), (173, 0.014710042379313922), (195, 0.011430482823601849), (220, 0.06385691997901066), (221, 0.1798951742468994), (244, 0.010535918368839705)]
[(22, 0.016797584585994722), (23, 0.018889342510485255), (36, 0.023863361204982593), (60, 0.04773169358500483), (68, 0.021510352799723527), (84, 0.19935809341839522), (101, 0.028674452115883462), (169, 0.02048300031220164), (220, 0.2329717282502984), (221, 0.1061165224415395)]
[(44, 0.01607217350184613), (84, 0.13078329648165526), (85, 0.030335848010421253), (93, 0.014211706939301677), (138, 0.10470320100113895), (151, 0.02057914965212648), (152, 0.01930297381677571), (200, 0.018707346581356258), (220, 0.024412555510165454), (221, 0.2227719667622663),

[(9, 0.017803993958233545), (74, 0.01694026525232577), (79, 0.015110626087177387), (84, 0.16727023361563456), (87, 0.04158751258792904), (91, 0.03600122076656171), (96, 0.0170532744149395), (132, 0.11380488050714208), (137, 0.015202335111634947), (142, 0.014202460400227358), (159, 0.019033585139078567), (172, 0.052567593948112716), (220, 0.04157683486635388), (221, 0.14229288867669937), (228, 0.01778643911258368)]
[(8, 0.011994989553015225), (27, 0.016274762283589195), (58, 0.029366752098552048), (68, 0.09117561565541626), (75, 0.01863645658381619), (84, 0.1291667566108416), (85, 0.06563608786808864), (117, 0.08388524528767234), (132, 0.012928232869323196), (149, 0.013233434078265112), (169, 0.03273037492965544), (207, 0.013982435115639224), (220, 0.02765708089114214), (221, 0.1769737836754215), (246, 0.02345390646880134)]
[(2, 0.010620010734590528), (5, 0.010000089691744585), (23, 0.058421930589432534), (26, 0.05019014022056031), (33, 0.02042573738551748), (61, 0.010824342892429993), 

In [23]:
avg_precision[1:]

[0.12705882352941178,
 0.15676470588235294,
 0.1469607843137255,
 0.1322794117647059,
 0.11952941176470591,
 0.10612745098039214,
 0.1049159663865546,
 0.0967279411764706,
 0.09140522875816995,
 0.09305882352941176]

In [14]:
clean_text=list(word_tokenize(cleantext(test_abs['abstract'].iloc[1]))) 
bow_text=dictionary.doc2bow(clean_text)
t=tfidf[bow_text]
lda_model[t]

NameError: name 'test_abs' is not defined

In [92]:
avg_precision[1:]

[0.07852941176470588,
 0.11764705882352941,
 0.13725490196078433,
 0.12257352941176472,
 0.11570588235294117,
 0.10622549019607845,
 0.09386554621848738,
 0.09316176470588236,
 0.0838888888888889,
 0.08532352941176473]

## Bag of Words Approach

In [32]:
reviewer_vec_bow={}
n=len(dictionary)
for sno,name in enumerate(review_lt):
    samp_df=data_df[data_df['author']==name]
    index_lt=samp_df.index
    wts=np.zeros([n], dtype = float)
    ctr=0
    #print(index_lt)
    for ind in index_lt:
        ctr+=1
        text=samp_df['title'][ind]+' '+samp_df['abstract'][ind]
        clean_text=list(word_tokenize(cleantext(text))) 
        bow_text=dictionary.doc2bow(clean_text)
        t=tfidf[bow_text]
        for ind,wt in t:
            wts[ind]+=wt
    wts=wts/ctr
    reviewer_vec_bow[sno]=wts

In [33]:
def predict_bow(text,reviewer_vec_bow):
    clean_text=list(word_tokenize(cleantext(text))) 
    bow_text=dictionary.doc2bow(clean_text)
    tfidf_text=tfidf[bow_text]
    
    n=len(dictionary)

    bow_vec=np.zeros(n, dtype = float)
    for ind,wt in tfidf_text:
        bow_vec[ind]=wt
        
    #print(topic)
    
    reviewer_scr=[]
    index_lt=list(reviewer_vec_bow.keys())
    for index in index_lt:
        val=1-cosine(bow_vec,reviewer_vec_bow[index])
        if np.isnan(val):
            val=0
        reviewer_scr.append([index+1,val])
        
    reviewer_scr.sort(key=lambda x: x[1],reverse=True)
    
    pred=[]
    for i in range(10):
        pred.append(reviewer_scr[i][0])
        
    return pred

In [60]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avg_precision=[0]*11
valid_file=list(judge.keys())
for i in range(n):
    if i in valid_file:        
        res=predict_bow(test_abs['abstract'].iloc[i],reviewer_vec_bow)
        #print(res,i)
        #print('////////////')
        for k in range(1,11):
            avg_precision[k]+=precision_k(res,judge,k,i)
            #print(precision_k(res,judge,k,i),i)
        ctr+=1
avg_precision=[ele/ctr for ele in avg_precision]

In [61]:
avg_precision[1:]

[0.47058823529411764,
 0.39705882352941174,
 0.3529411764705882,
 0.35294117647058826,
 0.35294117647058815,
 0.3333333333333333,
 0.3025210084033614,
 0.2867647058823529,
 0.2712418300653595,
 0.25882352941176473]

In [140]:
predict_bow(test_abs['abstract'].iloc[103],reviewer_vec_bow)

[234, 9, 53, 108, 224, 251, 197, 23, 235, 225]

In [143]:
'''path1=cwd+sym+'Saved_Items'+sym
with open(path1+'reviewer_vec_bow.pkl','wb') as f:
    pickle.dump(reviewer_vec_bow,f)'''

In [28]:
num_topics=250

In [50]:
def reviewer_404(res,ground_tht,paper_no):
    actual=ground_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    output=[]
    for auth in res:
        if auth not in actual_auth:
            sim_scr=[0]*4
            count=[0]*4
            vec1=reviewer_vec_bow[auth-1]
            for i in range(n):
                val=1-cosine(vec1,reviewer_vec_bow[actual_auth[i]-1])
                if np.isnan(val):
                    val=0
                sim_scr[rating[i]]+=val
                count[rating[i]]+=1
            sim_scr=[sim_scr[i]/count[i] for i in range(4) if count[i]]
            max_scr=max(sim_scr)
            ind=sim_scr.index(max_scr)
            output.append([auth,ind])
            #print(sim_scr)
    return output

In [51]:
def precision_k_404(res,gnd_tht,k,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    soft_p=0
    for i in range(k):
        pred=res[i]
        for j in range(n):
            if actual_auth[j]==pred:
                if rating[j]==3:
                    soft_p+=1
                elif rating[j]==2:
                    soft_p+=0.67
                elif rating[j]==1:
                    soft_p+=0.33
                else:
                    break
    rating_gen=reviewer_404(res,gnd_tht,paper_no)
    auth_gen=[i[0] for i in rating_gen]
    rating_gen=[i[1] for i in rating_gen]
    n1=len(auth_gen)
    for i in range(k):
        pred=res[i]
        for j in range(n1):
            if auth_gen[j]==pred:
                if rating_gen[j]==3:
                    soft_p+=1
                elif rating_gen[j]==2:
                    soft_p+=0.67
                elif rating_gen[j]==1:
                    soft_p+=0.33
                else:
                    break
    soft_p=soft_p/k
    return soft_p

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
def diversity(res):
    div=0
    n=len(res)
    for i in range(n):
        for j in range(n):
            sim=cosine_similarity([reviewer_vec[res[i]-1],reviewer_vec[res[j]-1]])
            sim=sim[0][1]
            div+=1-sim
    div=div/(n*(n-1))
    return div

In [73]:
def KL(p,q):
    """ Epsilon is used here to avoid conditional code for
    checking that neither P nor Q is equal to 0. """
    epsilon = 0.00001
    
    # You may want to instead make copies to avoid changing the np arrays.
    vec1=reviewer_vec[p-1]
    vec2=reviewer_vec[q-1]
    
    vec1=np.array(vec1)
    v1sum=np.sum(vec1)
    vec2=np.array(vec2)
    v2sum=np.sum(vec2)

    vec1=np.divide(vec1,v1sum)
    vec2=np.divide(vec2,v2sum)
        
    vec1 = vec1+epsilon
    vec2 = vec2+epsilon
    
    if v1sum==0 or v2sum==0:
        return 0

    divergence = np.sum(vec1*np.log10(vec1/vec2))
    return divergence

def diversity(res):
    div=0
    n=len(res)
    for i in range(n):
        for j in range(n):
            div+=KL(res[i],res[j])
    div=div/(n*(n-1))
    return div
    

In [76]:
def precision_k_404(res,gnd_tht,k,paper_no):
    actual=gnd_tht[paper_no]
    actual_auth=[i[0] for i in actual]
    rating=[i[1] for i in actual]
    n=len(actual)
    soft_p=0
    for i in range(k):
        pred=res[i]
        for j in range(n):
            if actual_auth[j]==pred:
                if rating[j]==3:
                    soft_p+=1
                elif rating[j]==2:
                    soft_p+=0.67
                elif rating[j]==1:
                    soft_p+=0.33
                else:
                    break
    rating_gen=reviewer_404(res,gnd_tht,paper_no)
    auth_gen=[i[0] for i in rating_gen]
    rating_gen=[i[1] for i in rating_gen]
    n1=len(auth_gen)
    for i in range(k):
        pred=res[i]
        for j in range(n1):
            if auth_gen[j]==pred:
                if rating_gen[j]==3:
                    soft_p+=1
                elif rating_gen[j]==2:
                    soft_p+=0.67
                elif rating_gen[j]==1:
                    soft_p+=0.33
                else:
                    break
    soft_p=soft_p/k
    return soft_p

In [84]:
test_abs=pd.read_pickle('test_abs')
n=len(test_abs)
ctr=0
avg_precision=[0]*11
valid_file=list(judge.keys())
for i in range(n):
    if i in valid_file:        
        res=predict(test_abs['abstract'].iloc[i],reviewer_vec_blda)
        #print(res)
        for k in range(1,11):
            avg_precision[k]+=precision_k(res,judge,k,i)
        ctr+=1
avg_precision=[ele/ctr for ele in avg_precision]

In [85]:
avg_precision[1:]

[0.17647058823529413,
 0.16176470588235295,
 0.1372549019607843,
 0.1323529411764706,
 0.13529411764705884,
 0.1323529411764706,
 0.1260504201680672,
 0.125,
 0.12418300653594774,
 0.11764705882352944]